In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import missingno as msno
import pandas_profiling
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
#解决中文显示问题
plt.rcParams["font.sans-serif"] = [u"SimHei"]
plt.rcParams["axes.unicode_minus"] = False
#解决图中负号问题
plt.rcParams['axes.unicode_minus']=False

In [130]:
column_name=['id','target','card_type','netlong','owner_city','arpu','out_prdct_fee','prdct_fee','point_fee','myth_fee','brand','listing_date','listing_price','nettime','user_level','sex','family_client','group_client','music','expirationtime','video_member','other_tv','finance','bank','campus_client','migu_tv','age','adis','arpu_avg','myth_fee_avg','out_prdct_fee_avg','prdct_fee_avg','os','battery_capacity','storage','screen_size']

# names = ['id','target','card_type','netlong','owner_city','arpu','out_prdct_fee','prdct_fee','point_fee','myth_fee','brand','listing_date','listing_price','nettime','user_level','sex','family_client','group_client','music','expirationtime','video_member','other_tv','finance','bank','campus_client','migu_tv','age','adis','arpu_avg','myth_fee_avg','out_prdct_fee_avg','prdct_fee_avg']
original = pd.read_table('ZZ0702GuangFa_Samples100000_FeatureData.txt',names=column_name)
display(original.head(),original.shape)

,id,target,card_type,netlong,owner_city,arpu,out_prdct_fee,prdct_fee,point_fee,myth_fee,...,age,adis,arpu_avg,myth_fee_avg,out_prdct_fee_avg,prdct_fee_avg,os,battery_capacity,storage,screen_size
0,13013302938,1,0.0,NaN,联通联通,9.0,0.0,0.0,0.0,0.0,...,NaN,NaN,38.863332,0.0,NaN,0.0,4.0,4120mAh,64GB,6.0
1,13022062073,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13035901978,1,NaN,NaN,NaN,37.2,0.0,0.0,0.0,0.0,...,NaN,NaN,37.013334,0.0,NaN,0.0,NaN,NaN,NaN,NaN
3,13042151031,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3050mAh,16/64GB,5.0
4,13045173577,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(103513, 36)

In [131]:
display(original.target.value_counts())

0    92989
1    10524
Name: target, dtype: int64

In [132]:
sample_false=original[original['target']==0].sample(n=10000, replace=False,random_state=0, axis=None).reset_index(drop=True)
sample_true=original[original['target']==1].sample(n=10000, replace=False,random_state=0, axis=None).reset_index(drop=True)
combin_df=sample_true.append(sample_false)
combin_df.to_excel('d:\sample_model.xlsx',index=None)

In [133]:
sample_true.head()

,id,target,card_type,netlong,owner_city,arpu,out_prdct_fee,prdct_fee,point_fee,myth_fee,...,age,adis,arpu_avg,myth_fee_avg,out_prdct_fee_avg,prdct_fee_avg,os,battery_capacity,storage,screen_size
0,18746191488,1,1.0,37.0,黑龙江哈尔滨,45.32,6.48,29.0,0.0,0.0,...,NaN,1.0,45.320000,0.0,6.48,29.0,NaN,NaN,NaN,NaN
1,13769713610,1,0.0,NaN,云南曲靖,115.70,0.00,0.0,0.0,0.0,...,NaN,NaN,115.099998,0.0,NaN,0.0,4.0,2920mAH,16GB/32GB,5.0
2,13838040676,1,1.0,NaN,河南郑州,143.20,0.20,135.0,9951.0,0.0,...,NaN,1.0,148.810003,0.0,0.20,135.0,4.0,3000mAh,32GB,5.5
3,15945389676,1,0.0,30.0,黑龙江牡丹江,60.99,1.00,90.0,0.0,0.0,...,NaN,NaN,60.990002,0.0,1.00,90.0,4.0,3000mAh,16GB,5.7
4,13888246777,1,1.0,NaN,云南昆明,258.00,0.00,0.0,0.0,0.0,...,NaN,NaN,258.000000,0.0,NaN,0.0,3.0,1715mAh,16GB/64GB/128GB,4.7


In [123]:
import random
import pandas as pd
def create_column_randomsample(column_random_dict):
    total_column_dict={}
    total_column_df=[]
    for columns_name in column_random_dict:
        if columns_name not in total_column_dict:
            total_column_dict[columns_name]=[]        
        for value_keys in column_random_dict[columns_name]:
            value_keys_counts=int(column_random_dict[columns_name][value_keys]*100000)
            for index in range(value_keys_counts):
                total_column_dict[columns_name].append(value_keys)
            
    result_df=pd.DataFrame(total_column_dict)
    for columns in result_df.columns:
        result_df[columns]=result_df[columns].sample(frac=1).reset_index(drop=True)
    return result_df

def create_number_randomsample(number_random_dict):
    import numpy as np
    import random
    
    total_column_dict={}
    
    all_True_column_df={}
    all_False_column_df={}
    
    for columns_name in number_random_dict:
        number_random_dict_column_keyslist=list(number_random_dict[columns_name].keys())
        random.shuffle(number_random_dict_column_keyslist)        
        if columns_name not in total_column_dict:
            signle_True_column_df=[]
            for column_section_key in number_random_dict_column_keyslist:
                True_column_df=[]
                
                left_value=int(column_section_key.split('-')[0])
                right_value=int(column_section_key.split('-')[1])
                
                True_sample_number=int(100000*number_random_dict[columns_name][column_section_key]['false_ratio'])
                for value_number in range(True_sample_number):
                    True_column_df.append(random.randint(left_value,right_value))
                #print(left_value,right_value,True_column_df)
                signle_True_column_df+=(True_column_df)
            #print(signle_True_column_df)
            all_True_column_df[columns_name]=signle_True_column_df
            
    
    for columns_name in number_random_dict:
        number_random_dict_column_keyslist=list(number_random_dict[columns_name].keys())
        random.shuffle(number_random_dict_column_keyslist)        
        if columns_name not in total_column_dict:
            signle_False_column_df=[]
            for column_section_key in number_random_dict_column_keyslist:
                False_column_df=[]
                
                left_value=int(column_section_key.split('-')[0])
                right_value=int(column_section_key.split('-')[1])
                False_sample_number=int(100000*number_random_dict[columns_name][column_section_key]['true_ratio'])
                for value_number in range(False_sample_number):
                    False_column_df.append(random.randint(left_value,right_value))
                #print(left_value,right_value,True_column_df)
                signle_False_column_df+=(False_column_df)
            all_False_column_df[columns_name]=signle_False_column_df
            
    for columns_name in number_random_dict:       
        total_column_dict[columns_name]=all_True_column_df[columns_name]+all_False_column_df[columns_name]
        
    
    return pd.DataFrame(total_column_dict)
            
    
True_number_random_dict={
                        '年龄':{
                                 '18-18':{'false_ratio':0.01,'true_ratio':0.01},'19-25':{'false_ratio':0.08,'true_ratio':0.01},
                                 '25-35':{'false_ratio':0.5,'true_ratio':0.3},'35-45':{'false_ratio':0.3,'true_ratio':0.53},
                                 '45-60':{'false_ratio':0.1,'true_ratio':0.14},'60-90':{'false_ratio':0.01,'true_ratio':0.01},},
                        '近1个月交易次数':{
                                 '0-0':{'false_ratio':0.04,'true_ratio':0.03},'0-1':{'false_ratio':0.11,'true_ratio':0.05},
                                 '1-2':{'false_ratio':0.16,'true_ratio':0.10},'2-3':{'false_ratio':0.3,'true_ratio':0.15},
                                 '3-8':{'false_ratio':0.2,'true_ratio':0.21},'8-15':{'false_ratio':0.12,'true_ratio':0.3},
                                 '15-30':{'false_ratio':0.05,'true_ratio':0.12},'30-300':{'false_ratio':0.02,'true_ratio':0.04},}, 
                        '近1个月笔均交易金额':{
                                 '0-100':{'false_ratio':0.02,'true_ratio':0.01},'100-300':{'false_ratio':0.23,'true_ratio':0.18},
                                 '300-500':{'false_ratio':0.35,'true_ratio':0.25},'500-1000':{'false_ratio':0.21,'true_ratio':0.32},
                                 '1000-10000':{'false_ratio':0.12,'true_ratio':0.15},'10000-100000':{'false_ratio':0.07,'true_ratio':0.09},},                                 
                         '近3个月交易次数':{
                                 '0-0':{'false_ratio':0.04,'true_ratio':0.02},'0-3':{'false_ratio':0.11,'true_ratio':0.04},
                                 '3-6':{'false_ratio':0.16,'true_ratio':0.11},'6-10':{'false_ratio':0.2,'true_ratio':0.15},
                                 '10-25':{'false_ratio':0.3,'true_ratio':0.19},'25-50':{'false_ratio':0.12,'true_ratio':0.33},
                                 '50-100':{'false_ratio':0.05,'true_ratio':0.12},'100-1000':{'false_ratio':0.02,'true_ratio':0.04},}, 
                         '近3个月笔均交易金额':{
                                 '0-100':{'false_ratio':0.02,'true_ratio':0.01},'100-300':{'false_ratio':0.2,'true_ratio':0.17},
                                 '300-500':{'false_ratio':0.38,'true_ratio':0.24},'500-1000':{'false_ratio':0.23,'true_ratio':0.34},
                                 '1000-10000':{'false_ratio':0.11,'true_ratio':0.16},'10000-100000':{'false_ratio':0.06,'true_ratio':0.08},}, 
                         '近6个月交易次数':{
                                 '0-0':{'false_ratio':0.04,'true_ratio':0.01},'0-5':{'false_ratio':0.09,'true_ratio':0.05},
                                 '5-10':{'false_ratio':0.16,'true_ratio':0.10},'10-20':{'false_ratio':0.2,'true_ratio':0.15},
                                 '20-50':{'false_ratio':0.32,'true_ratio':0.23},'50-100':{'false_ratio':0.12,'true_ratio':0.31},
                                 '100-200':{'false_ratio':0.05,'true_ratio':0.11},'200-2000':{'false_ratio':0.02,'true_ratio':0.04},},
                         '近6个月笔均交易金额':{
                                 '0-100':{'false_ratio':0.02,'true_ratio':0.01},'100-300':{'false_ratio':0.19,'true_ratio':0.16},
                                 '300-500':{'false_ratio':0.36,'true_ratio':0.20},'500-1000':{'false_ratio':0.24,'true_ratio':0.37},
                                 '1000-10000':{'false_ratio':0.13,'true_ratio':0.17},'10000-100000':{'false_ratio':0.06,'true_ratio':0.09},}, 
                         '近12个月交易次数':{
                                 '0-0':{'false_ratio':0.04,'true_ratio':0.01},'0-10':{'false_ratio':0.08,'true_ratio':0.02},
                                 '10-20':{'false_ratio':0.15,'true_ratio':0.08},'20-50':{'false_ratio':0.21,'true_ratio':0.13},
                                 '50-100':{'false_ratio':0.33,'true_ratio':0.23},'100-200':{'false_ratio':0.12,'true_ratio':0.34},
                                 '200-400':{'false_ratio':0.05,'true_ratio':0.15},'400-4000':{'false_ratio':0.02,'true_ratio':0.04},},
    
                         '近12个月笔均交易金额':{
                                 '0-100':{'false_ratio':0.02,'true_ratio':0.01},'100-300':{'false_ratio':0.21,'true_ratio':0.14},
                                 '300-500':{'false_ratio':0.4,'true_ratio':0.22},'500-1000':{'false_ratio':0.21,'true_ratio':0.35},
                                 '1000-10000':{'false_ratio':0.12,'true_ratio':0.2},'10000-100000':{'false_ratio':0.04,'true_ratio':0.08},},
    
                         '近1个月人民币活期存款':{
                                 '0-5000':{'false_ratio':0.15,'true_ratio':0.06},'5000-10000':{'false_ratio':0.07,'true_ratio':0.07},
                                 '10000-30000':{'false_ratio':0.1,'true_ratio':0.1},'30000-50000':{'false_ratio':0.17,'true_ratio':0.14},
                                 '50000-80000':{'false_ratio':0.25,'true_ratio':0.33},'80000-150000':{'false_ratio':0.14,'true_ratio':0.14},
                                 '150000-500000':{'false_ratio':0.08,'true_ratio':0.12},'500000-5000000':{'false_ratio':0.04,'true_ratio':0.04},},
                         '近3个月人民币活期存款':{
                                 '0-5000':{'false_ratio':0.13,'true_ratio':0.05},'5000-10000':{'false_ratio':0.06,'true_ratio':0.07},
                                 '10000-30000':{'false_ratio':0.11,'true_ratio':0.09},'30000-50000':{'false_ratio':0.17,'true_ratio':0.16},
                                 '50000-80000':{'false_ratio':0.26,'true_ratio':0.35},'80000-150000':{'false_ratio':0.15,'true_ratio':0.14},
                                 '150000-500000':{'false_ratio':0.08,'true_ratio':0.1},'500000-5000000':{'false_ratio':0.04,'true_ratio':0.04},},
    
    
    
    
    
    

                        '近6个月人民币活期存款余额均值':{
                                                         '0-5000':{'false_ratio':0.14,'true_ratio':0.04},'5000-10000':{'false_ratio':0.08,'true_ratio':0.07},
                                                         '10000-30000':{'false_ratio':0.11,'true_ratio':0.09},'30000-50000':{'false_ratio':0.17,'true_ratio':0.16},
                                                         '50000-80000':{'false_ratio':0.25,'true_ratio':0.32},'80000-150000':{'false_ratio':0.14,'true_ratio':0.18},
                                                         '150000-500000':{'false_ratio':0.07,'true_ratio':0.1},
                                                         '500000-5000000':{'false_ratio':0.04,'true_ratio':0.04}},

                        '近12个月人民币活期存款余额均值':{
                                                         '0-5000':{'false_ratio':0.11,'true_ratio':0.03},'5000-10000':{'false_ratio':0.08,'true_ratio':0.08},
                                                         '10000-30000':{'false_ratio':0.11,'true_ratio':0.12},'30000-50000':{'false_ratio':0.18,'true_ratio':0.16},
                                                         '50000-80000':{'false_ratio':0.27,'true_ratio':0.33},'80000-150000':{'false_ratio':0.12,'true_ratio':0.14},
                                                         '150000-500000':{'false_ratio':0.09,'true_ratio':0.09},
                                                         '500000-5000000':{'false_ratio':0.04,'true_ratio':0.05}},

                        '近1个月人民币定期存款余额':{
                                                 '0-10000':{'false_ratio':0.05,'true_ratio':0.02},'10000-20000':{'false_ratio':0.09,'true_ratio':0.06},
                                                 '20000-50000':{'false_ratio':0.21,'true_ratio':0.18},'50000-80000':{'false_ratio':0.32,'true_ratio':0.28},
                                                 '80000-150000':{'false_ratio':0.18,'true_ratio':0.22},'150000-500000':{'false_ratio':0.09,'true_ratio':0.15},
                                                 '500000-1000000':{'false_ratio':0.04,'true_ratio':0.07},
                                                 '1000000-10000000':{'false_ratio':0.02,'true_ratio':0.02}},
                        '近3个月人民币定期存款余额均值':{
                                                         '0-10000':{'false_ratio':0.05,'true_ratio':0.02},'10000-20000':{'false_ratio':0.08,'true_ratio':0.07},
                                                         '20000-50000':{'false_ratio':0.2,'true_ratio':0.18},'50000-80000':{'false_ratio':0.31,'true_ratio':0.27},
                                                         '80000-150000':{'false_ratio':0.2,'true_ratio':0.23},'150000-500000':{'false_ratio':0.09,'true_ratio':0.14},
                                                         '500000-1000000':{'false_ratio':0.05,'true_ratio':0.07},
                                                        '1000000-10000000':{'false_ratio':0.02,'true_ratio':0.02}},

                        '客户级信用额度':{
                                           '2000-4000':{'false_ratio':0.06,'true_ratio':0},'4000-6000':{'false_ratio':0.14,'true_ratio':0.02},
                                           '6000-10000':{'false_ratio':0.12,'true_ratio':0.12},'10000-20000':{'false_ratio':0.05,'true_ratio':0.18},
                                           '20000-50000':{'false_ratio':0.02,'true_ratio':0.27},'50000-500000':{'false_ratio':0.01,'true_ratio':0.05},
                                            '999999999-999999999':{'false_ratio':0.6,'true_ratio':0.36}},

                        '近1个月个人贷款':{
                                        '0-0':{'false_ratio':0.3,'true_ratio':0.45},
                                        '0-20000':{'false_ratio':0.1,'true_ratio':0.15},
                                        '20000-25000':{'false_ratio':0.3,'true_ratio':0.2},
                                        '25000-50000':{'false_ratio':0.15,'true_ratio':0.1},
                                        '50000-100000':{'false_ratio':0.1,'true_ratio':0.05},
                                        '100000-500000':{'false_ratio':0.03,'true_ratio':0.03},
                                        '500000-5000000':{'false_ratio':0.02,'true_ratio':0.02},},

                        
                        '近12个月行内总资产均值':{
                                            '0-10000':{'false_ratio':0.13,'true_ratio':0.03},
                                            '10000-30000':{'false_ratio':0.09,'true_ratio':0.08},
                                            '30000-100000':{'false_ratio':0.21,'true_ratio':0.15},
                                            '100000-150000':{'false_ratio':0.31,'true_ratio':0.23},
                                            '150000-300000':{'false_ratio':0.13,'true_ratio':0.31},
                                            '300000-500000':{'false_ratio':0.07,'true_ratio':0.12},
                                            '500000-1000000':{'false_ratio':0.04,'true_ratio':0.06},
                                            '1000000-10000000':{'false_ratio':0.02,'true_ratio':0.02}},

                        '近1个月行内总资产':{
                                        '0-10000':{'false_ratio':0.15,'true_ratio':0.04},
                                        '10000-30000':{'false_ratio':0.12,'true_ratio':0.12},
                                        '30000-100000':{'false_ratio':0.2,'true_ratio':0.19},
                                        '100000-150000':{'false_ratio':0.30,'true_ratio':0.24},
                                        '150000-300000':{'false_ratio':0.12,'true_ratio':0.19},
                                        '300000-500000':{'false_ratio':0.08,'true_ratio':0.14},
                                        '500000-1000000':{'false_ratio':0.02,'true_ratio':0.06},
                                        '1000000-10000000':{'false_ratio':0.01,'true_ratio':0.02}},

                        '近12个月收入总额':{
                                        '0-50000':{'false_ratio':0.15,'true_ratio':0.08},
                                        '50000-60000':{'false_ratio':0.22,'true_ratio':0.15},
                                        '60000-85000':{'false_ratio':0.25,'true_ratio':0.18},
                                        '85000-120000':{'false_ratio':0.14,'true_ratio':0.2},
                                        '120000-180000':{'false_ratio':0.1,'true_ratio':0.25},
                                        '180000-240000':{'false_ratio':0.09,'true_ratio':0.09},
                                        '240000-360000':{'false_ratio':0.04,'true_ratio':0.04},
                                        '360000-3600000':{'false_ratio':0.01,'true_ratio':0.01},},



                        '近6个月人民币定期存款余额均值':{
                                                    '0-10000':{'false_ratio':0.05,'true_ratio':0.02},
                                                    '10000-20000':{'false_ratio':0.08,'true_ratio':0.07},
                                                    '20000-50000':{'false_ratio':0.20,'true_ratio':0.18},
                                                    '50000-80000':{'false_ratio':0.30,'true_ratio':0.27},
                                                    '80000-150000':{'false_ratio':0.20,'true_ratio':0.23},
                                                    '150000-500000':{'false_ratio':0.1,'true_ratio':0.14},
                                                    '500000-1000000':{'false_ratio':0.05,'true_ratio':0.07},
                                                    '1000000-1000000':{'false_ratio':0.02,'true_ratio':0.02},
                                                    },


                        '近12个月人民币定期存款余额均值':{
                                                    '0-10000':{'false_ratio':0.04,'true_ratio':0.03},
                                                    '10000-20000':{'false_ratio':0.08,'true_ratio':0.07},
                                                    '20000-50000':{'false_ratio':0.16,'true_ratio':0.14},
                                                    '50000-80000':{'false_ratio':0.35,'true_ratio':0.30},
                                                    '80000-150000':{'false_ratio':0.20,'true_ratio':0.22},
                                                    '150000-500000':{'false_ratio':0.1,'true_ratio':0.15},
                                                    '500000-1000000':{'false_ratio':0.05,'true_ratio':0.05},
                                                    '1000000-1000000':{'false_ratio':0.02,'true_ratio':0.04},
                                                    },


                        '征信认定年收入':{
                                    '0-20000':{'false_ratio':0.01,'true_ratio':0.01},
                                    '20000-50000':{'false_ratio':0.05,'true_ratio':0.05},
                                    '50000-80000':{'false_ratio':0.2,'true_ratio':0.1},
                                    '80000-120000':{'false_ratio':0.19,'true_ratio':0.14},
                                    '120000-200000':{'false_ratio':0.15,'true_ratio':0.3},
                                    '200000-500000':{'false_ratio':0.1,'true_ratio':0.1},
                                    '500000-1000000':{'false_ratio':0.05,'true_ratio':0.05},
                                    '1000000-10000000':{'false_ratio':0.01,'true_ratio':0.01},
                                    '999999999-999999999':{'false_ratio':0.24,'true_ratio':0.24},
                                    },


                        '近3个月平均收入':{
                                    '0-3000':{'false_ratio':0.08,'true_ratio':0.1},
                                    '3000-7000':{'false_ratio':0.12,'true_ratio':0.1},
                                    '7000-10000':{'false_ratio':0.35,'true_ratio':0.2},
                                    '10000-15000':{'false_ratio':0.20,'true_ratio':0.25},
                                    '15000-20000':{'false_ratio':0.12,'true_ratio':0.1},
                                    '20000-25000':{'false_ratio':0.1,'true_ratio':0.1},
                                    '25000-50000':{'false_ratio':0.02,'true_ratio':0.1},
                                    '50000-500000':{'false_ratio':0.01,'true_ratio':0.05},
                                    },

                        '近3个月平均支出':{
                                    '0-1000':{'false_ratio':0.05,'true_ratio':0.1},
                                    '1000-1200':{'false_ratio':0.15,'true_ratio':0.1},
                                    '1200-1500':{'false_ratio':0.30,'true_ratio':0.15},
                                    '1500-2000':{'false_ratio':0.25,'true_ratio':0.15},
                                    '2000-2500':{'false_ratio':0.12,'true_ratio':0.15},
                                    '2500-3000':{'false_ratio':0.08,'true_ratio':0.22},
                                    '3000-3500':{'false_ratio':0.04,'true_ratio':0.11},
                                    '3500-35000':{'false_ratio':0.01,'true_ratio':0.02},
                                    },

                        '近3个月平均净收入（收入-支出）':{
                                                    '0-2000':{'false_ratio':0.06,'true_ratio':0.06},
                                                    '2000-5000':{'false_ratio':0.16,'true_ratio':0.16},
                                                    '5000-8000':{'false_ratio':0.34,'true_ratio':0.20},
                                                    '8000-12000':{'false_ratio':0.19,'true_ratio':0.20},
                                                    '12000-15000':{'false_ratio':0.12,'true_ratio':0.19},
                                                    '15000-20000':{'false_ratio':0.08,'true_ratio':0.14},
                                                    '20000-40000':{'false_ratio':0.04,'true_ratio':0.04},
                                                    '40000-400000':{'false_ratio':0.01,'true_ratio':0.01},
                                                    },
                        
                        }


False_column_random_dict={'性别':{'0':0.56,'1':0.44},
                         '婚姻状况':{'D':0.07,'M':0.42,'S':0.35,'W':0.04,'O':0.12},
                         '学历':{'0':0.01,'1':0.03,'2':0.05,'3':0.15,'4':0.45,'5':0.15,'6':0.05,'7':0.05,'8':0.05,'9':0.01},
                         '居住情况':{'1':0.03,'2':0.03,'3':0.44,'4':0.1,'5':0.2,'6':0.1,'7':0.1},
                         '社会保障':{'1':0.7,'0':0.3},
                         '单位性质':{'00':0.07,'01':0.01,'02':0.3,'03':0.23,'04':0.01,'05':0.08,'06':0.1,'07':0.06,'08':0.07,
                                     '09':0.04,'10':0,'11':0.03},
                         '所属城市级别':{'1':0.25,'2':0.25,'3':0.15,'4':0.15,'5':0.1,'6':0.05,'7':0.05},
                         '客户风险承受能力等级':{'1':0.33,'2':0.23,'3':0.22,'4':0.13,'5':0.09},                       
                         
                        }
True_column_random_dict={'性别':{'0':0.68,'1':0.32},
                         '婚姻状况':{'D':0.05,'M':0.45,'S':0.35,'W':0.04,'O':0.11},
                         '学历':{'0':0,'1':0.01,'2':0.02,'3':0.1,'4':0.35,'5':0.25,'6':0.2,'7':0.05,'8':0.02,'9':0},
                         '居住情况':{'1':0.01,'2':0.01,'3':0.1,'4':0.3,'5':0.35,'6':0.2,'7':0.03},
                         '社会保障':{'1':0.9,'0':0.1},                         
                         '单位性质':{'00':0.15,'01':0.01,'02':0.38,'03':0.1,'04':0.01,'05':0.08,'06':0.1,'07':0.06,'08':0.07,
                                     '09':0.01,'10':0,'11':0.03},
                         '所属城市级别':{'1':0.35,'2':0.25,'3':0.15,'4':0.15,'5':0.05,'6':0.03,'7':0.02},
                         '客户风险承受能力等级':{'1':0.33,'2':0.33,'3':0.22,'4':0.1,'5':0.02},                
                                                
                        }



False_result_df=create_column_randomsample(False_column_random_dict)
False_result_df['target']=0
True_result_df=create_column_randomsample(True_column_random_dict)
True_result_df['target']=1

total_classify_df=False_result_df.append(True_result_df)
total_numerical_df=create_number_randomsample(True_number_random_dict)
total_classify_df.reset_index(drop=True,inplace=True)
total_numerical_df.reset_index(drop=True,inplace=True)

Resulet_combin_df_total=total_classify_df.merge(total_numerical_df,left_index=True, right_index=True)
Resulet_combin_df_total=Resulet_combin_df_total[['性别', '婚姻状况', '学历', '居住情况', '社会保障', '单位性质', '所属城市级别', '客户风险承受能力等级',
       '年龄', '近1个月交易次数', '近1个月笔均交易金额', '近3个月交易次数', '近3个月笔均交易金额',
       '近6个月交易次数', '近6个月笔均交易金额', '近12个月交易次数', '近12个月笔均交易金额', '近1个月人民币活期存款',
       '近3个月人民币活期存款', '近6个月人民币活期存款余额均值', '近12个月人民币活期存款余额均值', '近1个月人民币定期存款余额',
       '近3个月人民币定期存款余额均值', '客户级信用额度', '近1个月个人贷款', '近12个月行内总资产均值', '近1个月行内总资产',
       '近12个月收入总额', '近6个月人民币定期存款余额均值', '近12个月人民币定期存款余额均值', '征信认定年收入',
       '近3个月平均收入', '近3个月平均支出', '近3个月平均净收入（收入-支出）','target']]


Resulet_combin_df_sample=Resulet_combin_df_total[Resulet_combin_df_total['target']==1].sample(n=20000).append(Resulet_combin_df_total[Resulet_combin_df_total['target']==0].sample(frac=1))

Resulet_combin_df_sample.to_excel('d:\\上海兴业银行样本造数.xlsx',index=None)

In [124]:
total_numerical_df.to_excel('d:\\兴业连续变量.xlsx',index=None)

In [92]:
display(Resulet_combin_df_total.columns,Resulet_combin_df_total[Resulet_combin_df['target']==1].sample(n=20000).head())

Index(['性别', '婚姻状况', '学历', '居住情况', '社会保障', '单位性质', '所属城市级别', '客户风险承受能力等级',
       'target', '年龄', '近1个月交易次数', '近1个月笔均交易金额', '近3个月交易次数', '近3个月笔均交易金额',
       '近6个月交易次数', '近6个月笔均交易金额', '近12个月交易次数', '近12个月笔均交易金额', '近1个月人民币活期存款',
       '近3个月人民币活期存款', '近6个月人民币活期存款余额均值', '近12个月人民币活期存款余额均值', '近1个月人民币定期存款余额',
       '近3个月人民币定期存款余额均值', '客户级信用额度', '近1个月个人贷款', '近12个月行内总资产均值', '近1个月行内总资产',
       '近12个月收入总额', '近6个月人民币定期存款余额均值', '近12个月人民币定期存款余额均值', '征信认定年收入',
       '近3个月平均收入', '近3个月平均支出', '近3个月平均净收入（收入-支出）'],
      dtype='object')

,性别,婚姻状况,学历,居住情况,社会保障,单位性质,所属城市级别,客户风险承受能力等级,target,年龄,...,近1个月个人贷款,近12个月行内总资产均值,近1个月行内总资产,近12个月收入总额,近6个月人民币定期存款余额均值,近12个月人民币定期存款余额均值,征信认定年收入,近3个月平均收入,近3个月平均支出,近3个月平均净收入（收入-支出）
84200,男,S,4,6,0,02,1,2,1,69,...,67766,89593,230335,84930,145674,21885,999999999,9240,1928,23737
70522,男,S,4,6,1,05,1,2,1,30,...,1636909,79146,51336,404720,62771,1000000,999999999,9882,2936,19151
582,男,S,6,5,0,02,2,1,1,19,...,678410,6295,135632,99330,9283,345939,635480,112850,1382,87046
67137,男,S,7,4,1,03,1,1,1,30,...,2718967,231844,84116,52908,68066,858703,999999999,9582,2987,21313
34071,男,S,5,6,0,01,1,2,1,87,...,10452,149693,4514,43783,25378,51326,110532,19763,2438,12402


In [68]:
display(False_result_df.head(),True_result_df.head())

,性别,婚姻状况,学历,征信认定年收入,社会保障,单位性质,客户评级结果,手机品牌,客户风险承受能力等级,客户级信用额度,零售潜在客户,近12月是否贷款,target
0,男,未婚,大专,5万,1,私营企业,C4,小米,较低,50000,否,是,0
1,男,未婚,高中,10万,1,股份制企业,C2,荣耀,基本,10000,否,否,0
2,男,已婚,大专,5万,1,股份制企业,C3,荣耀,基本,10000,是,否,0
3,男,已婚,研究生,5万,1,外商投资企业,C4,小米,较低,5000,否,否,0
4,女,未婚,高中中专,10万,1,私营企业,C2,华为,基本,30000,是,否,0


,性别,婚姻状况,学历,征信认定年收入,社会保障,单位性质,客户评级结果,手机品牌,客户风险承受能力等级,客户级信用额度,零售潜在客户,近12月是否贷款,target
0,男,未婚,初中,10万,1,私营企业,C3,苹果,基本,50000,否,是,1
1,男,已婚,大专,5万,1,外商投资企业,C3,苹果,基本,20000,否,否,1
2,男,未婚,本科,30万,1,国有企业,C4,OPPO,中等,20000,是,是,1
3,男,已婚,研究生,5万,1,私营企业,C4,vivo,较低,20000,否,否,1
4,男,已婚,研究生,10万,1,私营企业,C5,华为,中等,50000,否,否,1


In [65]:
display(Resulet_combin_df.shape,Resulet_combin_df.head())

(200000, 39)

,性别,婚姻状况,学历,征信认定年收入_x,社会保障,单位性质,客户评级结果,手机品牌,客户风险承受能力等级,客户级信用额度_x,...,近1个月个人贷款,近12个月行内总资产均值,近1个月行内总资产,近12个月收入总额,近6个月人民币定期存款余额均值,近12个月人民币定期存款余额均值,征信认定年收入_y,近3个月平均收入,近3个月平均支出,近3个月平均净收入（收入-支出）
0,女,未婚,高中中专,10万,1,外商投资企业,C1,小米,较低,5000,...,1829810,126076,111308,5447,7782,1000000,76044,3127,2843,26612
0,男,已婚,初中,5万,1,外商投资企业,C3,苹果,中等,20000,...,1829810,126076,111308,5447,7782,1000000,76044,3127,2843,26612
1,女,已婚,高中中专,10万,0,股份制企业,C3,小米,较低,10000,...,1412901,136978,132059,16073,9280,1000000,76589,3392,2986,24731
1,女,未婚,大专,5万,0,私营企业,C4,OPPO,基本,50000,...,1412901,136978,132059,16073,9280,1000000,76589,3392,2986,24731
2,女,已婚,高中中专,5万,1,股份制企业,C1,vivo,基本,10000,...,1622047,119023,135160,23569,7482,1000000,72220,2165,2838,26745


In [59]:
display(False_result_df.head(),total_classify_df.head(),True_result_df.tail(),total_classify_df.tail())

,性别,婚姻状况,学历,征信认定年收入,社会保障,单位性质,客户评级结果,手机品牌,客户风险承受能力等级,客户级信用额度,零售潜在客户,近12月是否贷款
0,女,未婚,大专,5万,1,外商投资企业,C2,小米,基本,20000,否,是
1,女,已婚,高中中专,5万,0,私营企业,C1,荣耀,较低,20000,否,是
2,女,已婚,大专,10万,0,外商投资企业,C2,华为,中等,10000,否,否
3,女,已婚,高中中专,5万,0,股份制企业,C2,小米,较低,10000,否,否
4,男,已婚,高中中专,10万,0,股份制企业,C2,vivo,较低,10000,是,否


,性别,婚姻状况,学历,征信认定年收入,社会保障,单位性质,客户评级结果,手机品牌,客户风险承受能力等级,客户级信用额度,零售潜在客户,近12月是否贷款
0,女,未婚,大专,5万,1,外商投资企业,C2,小米,基本,20000,否,是
1,女,已婚,高中中专,5万,0,私营企业,C1,荣耀,较低,20000,否,是
2,女,已婚,大专,10万,0,外商投资企业,C2,华为,中等,10000,否,否
3,女,已婚,高中中专,5万,0,股份制企业,C2,小米,较低,10000,否,否
4,男,已婚,高中中专,10万,0,股份制企业,C2,vivo,较低,10000,是,否


,性别,婚姻状况,学历,征信认定年收入,社会保障,单位性质,客户评级结果,手机品牌,客户风险承受能力等级,客户级信用额度,零售潜在客户,近12月是否贷款
99995,男,已婚,大专,40万,1,私营企业,C3,小米,基本,50000,否,是
99996,女,未婚,大专,5万,1,外商投资企业,C4,荣耀,基本,20000,是,否
99997,女,已婚,大专,10万,1,外商投资企业,C3,OPPO,基本,50000,是,否
99998,女,已婚,大专,10万,1,外商投资企业,C4,苹果,较低,50000,是,是
99999,男,已婚,大专,10万,1,股份制企业,C3,vivo,基本,10000,是,是


,性别,婚姻状况,学历,征信认定年收入,社会保障,单位性质,客户评级结果,手机品牌,客户风险承受能力等级,客户级信用额度,零售潜在客户,近12月是否贷款
99995,男,已婚,大专,40万,1,私营企业,C3,小米,基本,50000,否,是
99996,女,未婚,大专,5万,1,外商投资企业,C4,荣耀,基本,20000,是,否
99997,女,已婚,大专,10万,1,外商投资企业,C3,OPPO,基本,50000,是,否
99998,女,已婚,大专,10万,1,外商投资企业,C4,苹果,较低,50000,是,是
99999,男,已婚,大专,10万,1,股份制企业,C3,vivo,基本,10000,是,是


In [47]:
#True_result_df.shape()
total_df.head()

,年龄,近1个月交易次数,近1个月笔均交易金额,近3个月交易次数,近3个月笔均交易金额,近6个月交易次数,近6个月笔均交易金额,近12个月交易次数,近12个月笔均交易金额,近1个月人民币活期存款,...,近1个月个人贷款,近12个月行内总资产均值,近1个月行内总资产,近12个月收入总额,近6个月人民币定期存款余额均值,近12个月人民币定期存款余额均值,征信认定年收入,近3个月平均收入,近3个月平均支出,近3个月平均净收入（收入-支出）
0,65,8,236,775,592,184,226,122,778,140597,...,22637,3421,282852,2965888,654147,321262,108817,8459,1426,580478
1,65,3,149,803,661,108,161,122,751,101894,...,37904,6686,228287,1427034,591676,236010,86954,9936,1328,899123
2,68,5,220,868,929,173,182,188,980,109067,...,43135,2077,282867,2260092,712199,242831,93481,9244,1462,417861
3,70,5,247,432,758,128,131,133,558,141182,...,24243,4467,201001,2235795,751191,222176,98540,7019,1420,879418
4,62,8,107,718,816,161,200,101,799,101291,...,48265,3217,259453,1901095,954362,163385,106441,9390,1350,140804


In [36]:
total_df.head()

,年龄,近1个月交易次数,近1个月笔均交易金额,近3个月交易次数,近3个月笔均交易金额,近6个月交易次数,近6个月笔均交易金额,近12个月交易次数,近12个月笔均交易金额,近1个月人民币活期存款,...,近1个月个人贷款,近12个月行内总资产均值,近1个月行内总资产,近12个月收入总额,近6个月人民币定期存款余额均值,近12个月人民币定期存款余额均值,征信认定年收入,近3个月平均收入,近3个月平均支出,近3个月平均净收入（收入-支出）
0,27,1,94,34,7853,886,66134,3,146,8751,...,2743081,107077,135617,158016,859239,13739,501743,38603,2500,183926
1,26,1,28,29,1917,1999,63161,2,174,9732,...,1107996,139670,145622,177851,589064,14032,975132,34732,2123,414954
2,27,2,32,44,9667,316,28266,9,163,5204,...,2788589,100820,149984,148057,674287,12326,628014,32343,2314,183618
3,26,1,73,47,5556,934,50342,0,240,9138,...,1688877,137135,102210,143818,664788,15863,726177,33273,2079,279426
4,25,2,92,48,3441,801,26665,5,174,9480,...,2904329,142293,137022,165602,609329,18874,513511,40285,2084,490758


In [37]:
for columns in total_df:
    print(columns,total_df[columns].value_counts())
#pandas_profiling.ProfileReport(total_df[['近1个月交易次数']])

年龄 35     9898
25     7674
45     7179
40     5224
42     5167
39     5137
37     5131
44     5047
43     5042
38     5032
36     5029
41     4979
27     4813
33     4773
26     4741
29     4711
32     4699
30     4682
31     4672
28     4659
34     4654
60     4095
24     2950
22     2926
20     2890
18     2864
19     2834
21     2821
23     2783
70     2336
       ... 
80      205
74      204
94      202
90      201
71      199
104     196
79      195
118     195
82      194
111     193
77      189
110     187
15      134
13      134
0       122
1       120
16      118
11      115
2       115
7       114
4       114
6       111
9       110
5       107
10      106
17      104
8       101
14       97
3        91
12       87
Name: 年龄, Length: 121, dtype: int64
近1个月交易次数 2      35439
3      29301
1      20983
0      15038
8      12141
6       6961
5       6884
4       6876
7       6689
15      6215
10      5311
14      5292
9       5268
12      5249
11      5237
13      5151
26      1108

客户级信用额度 999999999    96000
7174            19
5350            19
4613            18
5695            18
5292            18
5322            17
5066            17
4450            17
4749            16
4603            16
5180            16
4609            16
5917            16
5328            16
8728            16
4161            16
5877            16
5661            16
5594            16
5605            16
6000            16
4815            16
8818            16
4529            16
5145            15
4650            15
5056            15
9396            15
5635            15
             ...  
314039           1
195843           1
37564            1
33470            1
35519            1
434938           1
29472            1
99564            1
27427            1
44361            1
23205            1
36173            1
43579            1
11609            1
21846            1
30034            1
6701             1
27985            1
25936            1
40271            1
37436            1
2525

In [ ]:
                         

true_result_df=create_column_randomsample(True_column_random_dict)
true_result_number_df=create_number_randomsample(True_number_random_dict)

True_combin_df=true_result_df.merge(true_result_number_df,left_index=True, right_index=True)



#True_combin_df=True_combin_df.merge(pd.DataFrame(sample_true[['id','owner_city']]),left_index=True, right_index=True)
#True_combin_df.rename(columns={'owner_city':'籍贯'},inplace=True)

                            
                                
'''
True_combin_df['target']=1


False_result_df=create_column_randomsample(False_column_random_dict)
False_result_number_df=create_number_randomsample(False_number_random_dict)
False_combin_df=False_result_df.merge(False_result_number_df,left_index=True, right_index=True)
#False_combin_df=False_combin_df.merge(pd.DataFrame(sample_false[['id','owner_city']]),left_index=True, right_index=True)
#False_combin_df.rename(columns={'owner_city':'籍贯'},inplace=True)
False_combin_df['target']=0

total_comple=True_combin_df.append(False_combin_df).reset_index(drop=True)

#total_comple=total_comple[['id','性别','年龄','籍贯','婚姻状况','学历','征信认定年收入','社会保障','客户级信用额度','美元账户额度','CRM评级结果','客户评级结果','借记卡层级','附卡卡数','存款期限','是否为低保户','受否贫困户','行业','职业','零售潜在客户','理财产品风险等级','手机品牌','基金风险等级','理财产品是否保本','客户风险承受能力等级','电子渠道购买标志','贷款放款本金','账户余额','贷款期数','剩余贷款本金','购买金额_事件表','数量_事件表','可用余额_事件表','分期金额_事件表','贷款期限_事件表','未还本金_事件表','偿还金额_事件表','金额_事件表','交易金额_交易日志表','产品万份收益_基金产品','车产价值_字典列表','房产价值_字典列表','交易金额_交易往来','存期_交易账务','交易金额_交易账务','行内总资产_资产负债','行内总负债_资产负债','本外币存款_资产负债','人民币活期存款_资产负债','人民币定期存款_资产负债','本外币理财_资产负债','人民币理财_资产负债','万利宝理财_资产负债','保本型理财_人民币_资产负债','结构性存款_人民币_资产负债','兴业宝资产_资产负债','综合金融资产_资产负债','券商集合理财_资产负债','保险_资产负债','个人贷款_资产负债','个人购房按揭贷款_资产负债','个人委托贷款_资产负债','个人消费贷款_资产负债','target']]
total_comple=total_comple[['性别','年龄','婚姻状况','学历','开户日期','征信认定年收入','社会保障','客户级信用额度','美元账户额度','CRM评级结果','客户评级结果','借记卡层级','附卡卡数','存款期限','是否为低保户','受否贫困户','行业','职业','零售潜在客户','理财产品风险等级','手机品牌','基金风险等级','理财产品是否保本','客户风险承受能力等级','电子渠道购买标志','贷款放款本金','账户余额','贷款期数','剩余贷款本金','购买金额_事件表','数量_事件表','可用余额_事件表','分期金额_事件表','贷款期限_事件表','未还本金_事件表','偿还金额_事件表','金额_事件表','交易金额_交易日志表','产品万份收益_基金产品','车产价值_字典列表','房产价值_字典列表','交易金额_交易往来','存期_交易账务','交易金额_交易账务','行内总资产_资产负债','行内总负债_资产负债','本外币存款_资产负债','人民币活期存款_资产负债','人民币定期存款_资产负债','本外币理财_资产负债','人民币理财_资产负债','万利宝理财_资产负债','保本型理财_人民币_资产负债','结构性存款_人民币_资产负债','兴业宝资产_资产负债','综合金融资产_资产负债','券商集合理财_资产负债','保险_资产负债','个人贷款_资产负债','个人购房按揭贷款_资产负债','个人委托贷款_资产负债','个人消费贷款_资产负债','target']]


# 生成随机日期函数：
import datetime
def create_random_date(rowdata):
    #print (datetime.datetime.now()+datetime.timedelta(days=-1)).strftime("%Y-%m-%d %H:%M:%S")
    #print(datetime.datetime.now()+datetime.timedelta(days=rowdata['开户日期']).strftime("%Y-%m-%d"))
    print((datetime.datetime.now()-datetime.timedelta(days=rowdata['开户日期'])).strftime("%Y-%m-%d"))
    return (datetime.datetime.now()-datetime.timedelta(days=rowdata['开户日期'])).strftime("%Y-%m-%d")
total_comple['开户日期']=total_comple.apply(create_random_date,axis=1)
'''

In [135]:
total_comple.head()

,性别,年龄,婚姻状况,学历,开户日期,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,...,结构性存款_人民币_资产负债,兴业宝资产_资产负债,综合金融资产_资产负债,券商集合理财_资产负债,保险_资产负债,个人贷款_资产负债,个人购房按揭贷款_资产负债,个人委托贷款_资产负债,个人消费贷款_资产负债,target
0,男,12,未婚,大专,1999-08-15,5万,0,20000,0,核心,...,49405,23432,25468,10041,4184,78639,376452,12277,5605,1
1,女,25,已婚,初中中专,2019-09-17,10万,1,50000,0,普通,...,39884,18805,14512,7880,3957,80815,379467,11764,3858,1
2,男,-10,未婚,大专,2015-03-27,5万,0,10000,0,普通,...,50171,11834,11639,9109,4451,66963,295913,18028,5434,1
3,男,20,已婚,大专,2002-10-13,10万,1,30000,0,核心,...,44882,19946,9732,10432,4529,70032,510295,14891,4267,1
4,男,19,已婚,大专,2003-02-18,5万,0,50000,0,普通,...,44677,15478,12109,9432,4324,72221,343757,18678,5915,1


In [136]:
# 对样本数据进行处理，将负值修改为0

random_columns_list=['贷款放款本金','账户余额','贷款期数','剩余贷款本金','购买金额_事件表','数量_事件表','可用余额_事件表','分期金额_事件表','贷款期限_事件表','未还本金_事件表','偿还金额_事件表','金额_事件表','交易金额_交易日志表','产品万份收益_基金产品','车产价值_字典列表','房产价值_字典列表','交易金额_交易往来','存期_交易账务','交易金额_交易账务','行内总资产_资产负债','行内总负债_资产负债','本外币存款_资产负债','人民币活期存款_资产负债','人民币定期存款_资产负债','本外币理财_资产负债','人民币理财_资产负债','万利宝理财_资产负债','保本型理财_人民币_资产负债','结构性存款_人民币_资产负债','兴业宝资产_资产负债','综合金融资产_资产负债','券商集合理财_资产负债','保险_资产负债','个人贷款_资产负债','个人购房按揭贷款_资产负债','个人委托贷款_资产负债','个人消费贷款_资产负债']


def deal_zero_value(rowdata,column):
    if rowdata[column]<0:
        return 0
    else:
        return rowdata[column]
    
    
for random_colum in random_columns_list:    
    total_comple[random_colum]=total_comple.apply(deal_zero_value,axis=1, args = (random_colum,))


In [137]:
display(True_combin_df.head(),False_combin_df.head(),total_comple.tail())
print(total_comple.shape,True_combin_df.shape,False_combin_df.shape,total_comple.columns)

,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,...,兴业宝资产_资产负债,综合金融资产_资产负债,券商集合理财_资产负债,保险_资产负债,个人贷款_资产负债,个人购房按揭贷款_资产负债,个人委托贷款_资产负债,个人消费贷款_资产负债,开户日期,target
0,男,未婚,大专,5万,0,20000,0,核心,C5,金卡,...,23432,25468,10041,4184,78639,376452,12277,5605,7446.714355,1
1,女,已婚,初中中专,10万,1,50000,0,普通,C1,金卡,...,18805,14512,7880,3957,80815,379467,11764,3858,108.866684,1
2,男,未婚,大专,5万,0,10000,0,普通,C3,普卡,...,11834,11639,9109,4451,66963,295913,18028,5434,1744.237549,1
3,男,已婚,大专,10万,1,30000,0,核心,C2,白金,...,19946,9732,10432,4529,70032,510295,14891,4267,6292.313477,1
4,男,已婚,大专,5万,0,50000,0,普通,C4,普卡,...,15478,12109,9432,4324,72221,343757,18678,5915,6164.348145,1


,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,...,兴业宝资产_资产负债,综合金融资产_资产负债,券商集合理财_资产负债,保险_资产负债,个人贷款_资产负债,个人购房按揭贷款_资产负债,个人委托贷款_资产负债,个人消费贷款_资产负债,开户日期,target
0,男,已婚,小学,40万,1,10000,0,普通,C3,普卡,...,11747,9048,9531,929,47535,213117,6595,22222,1088.429199,0
1,女,未婚,大专,5万,1,10000,0,准核心,C2,白金,...,7289,12950,8118,861,48017,162023,11036,9673,1220.908081,0
2,女,已婚,高中中专,5万,0,20000,0,普通,C4,普卡,...,15520,9893,7520,767,53450,151895,10159,-3269,957.995117,0
3,男,未婚,高中中专,5万,1,10000,0,普通,C3,普卡,...,8076,9410,4284,487,45945,103304,6625,-11558,1077.512817,0
4,女,未婚,高中中专,5万,1,10000,0,普通,C3,白金,...,13992,13881,4240,786,55331,176267,10152,-4469,689.687500,0


,性别,年龄,婚姻状况,学历,开户日期,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,...,结构性存款_人民币_资产负债,兴业宝资产_资产负债,综合金融资产_资产负债,券商集合理财_资产负债,保险_资产负债,个人贷款_资产负债,个人购房按揭贷款_资产负债,个人委托贷款_资产负债,个人消费贷款_资产负债,target
99995,男,29,已婚,高中中专,2017-03-30,10万,1,20000,0,普通,...,41915,15396,9325,5536,906,50827,82965,11304,9481,0
99996,男,37,未婚,高中,2017-10-11,10万,1,5000,0,核心,...,25032,24468,12483,5174,1037,48866,183879,8211,0,0
99997,男,25,未婚,高中中专,2017-03-30,10万,0,10000,0,普通,...,27005,18522,9303,6758,721,54899,182742,10197,8090,0
99998,女,29,未婚,高中中专,2016-12-19,10万,1,50000,0,普通,...,45068,12262,5797,4672,1036,52021,243886,9382,613,0
99999,男,9,未婚,大专,2017-05-14,5万,0,5000,0,准核心,...,35195,13049,12545,4444,1359,47716,170418,9210,19285,0


(100000, 63) (50000, 63) (50000, 63) Index(['性别', '年龄', '婚姻状况', '学历', '开户日期', '征信认定年收入', '社会保障', '客户级信用额度',
       '美元账户额度', 'CRM评级结果', '客户评级结果', '借记卡层级', '附卡卡数', '存款期限', '是否为低保户',
       '受否贫困户', '行业', '职业', '零售潜在客户', '理财产品风险等级', '手机品牌', '基金风险等级', '理财产品是否保本',
       '客户风险承受能力等级', '电子渠道购买标志', '贷款放款本金', '账户余额', '贷款期数', '剩余贷款本金',
       '购买金额_事件表', '数量_事件表', '可用余额_事件表', '分期金额_事件表', '贷款期限_事件表', '未还本金_事件表',
       '偿还金额_事件表', '金额_事件表', '交易金额_交易日志表', '产品万份收益_基金产品', '车产价值_字典列表',
       '房产价值_字典列表', '交易金额_交易往来', '存期_交易账务', '交易金额_交易账务', '行内总资产_资产负债',
       '行内总负债_资产负债', '本外币存款_资产负债', '人民币活期存款_资产负债', '人民币定期存款_资产负债',
       '本外币理财_资产负债', '人民币理财_资产负债', '万利宝理财_资产负债', '保本型理财_人民币_资产负债',
       '结构性存款_人民币_资产负债', '兴业宝资产_资产负债', '综合金融资产_资产负债', '券商集合理财_资产负债', '保险_资产负债',
       '个人贷款_资产负债', '个人购房按揭贷款_资产负债', '个人委托贷款_资产负债', '个人消费贷款_资产负债', 'target'],
      dtype='object')


In [138]:
total_comple.to_excel('d:/上海兴业银行样本数据集.xlsx',index=None)


In [ ]:
# 生成随机日期函数：
import
def create_random_date(rowdata):
    print(rowdata.)
    

In [8]:
True_combin_df.head()

,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,...,存款期限,是否为低保户,受否贫困户,年龄,贷款放款本金,账户余额,贷款期数,剩余贷款本金,id,籍贯
0,女,未婚,本科,5万,1,20000,小于0,核心,C4,普卡,...,6个月,否,否,10,539739,-7114,123,404910,13013302938,联通联通
1,男,已婚,大专,20万,1,20000,大于0,核心,C4,金卡,...,1年,否,否,48,807197,74019,63,404393,13022062073,NaN
2,男,已婚,高中中专,5万,1,10000,小于0,核心,C6,白金,...,1年,否,否,11,679981,216705,117,63766,13035901978,NaN
3,男,已婚,高中,5万,1,50000,小于0,核心,C1,白金,...,5年,否,否,30,498424,185155,201,330451,13042151031,NaN
4,男,已婚,大专,10万,1,50000,小于0,核心,C1,普卡,...,5年,否,否,12,511549,115947,-6,326409,13045173577,NaN


In [21]:
combin_df[['id','性别','籍贯','婚姻状况','学历','征信认定年收入','社会保障','客户级信用额度','美元账户额度','CRM评级结果','客户评级结果','借记卡层级','账户余额','剩余贷款本金']].to_excel('d:\上海兴业银行样本造数.xlsx',index=None)

In [164]:
display(true_result_df.head(),true_result_number_df.head())

,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,附卡卡数,存款期限,是否为低保户,受否贫困户
0,男,已婚,大专,5万,1,30000,小于0,普通,C3,黑金,4,1年,否,否
1,男,已婚,高中,10万,1,30000,小于0,准核心,C3,普卡,0,3个月,否,否
2,女,已婚,本科,5万,0,20000,小于0,普通,C4,普卡,3,1年,否,否
3,男,已婚,大专,10万,1,20000,小于0,普通,C3,金卡,1,2年,否,否
4,女,已婚,小学,5万,1,20000,小于0,普通,C3,白金,0,5年,否,否


,年龄,贷款放款本金,账户余额,贷款期数,剩余贷款本金
0,37,691320,8,317,445082
1,15,645061,29,49,196353
2,19,579867,29,157,368844
3,10,667989,34,33,329207
4,30,566405,51,118,330431


In [12]:
pd.DataFrame

,id,target,card_type,netlong,owner_city,arpu,out_prdct_fee,prdct_fee,point_fee,myth_fee,...,age,adis,arpu_avg,myth_fee_avg,out_prdct_fee_avg,prdct_fee_avg,os,battery_capacity,storage,screen_size
97032,18746191488,1,1.0,37.0,黑龙江哈尔滨,45.32,6.48,29.0,0.0,0.0,...,NaN,1.0,45.320000,0.0,6.480000,29.000000,NaN,NaN,NaN,NaN
29750,13769713610,1,0.0,NaN,云南曲靖,115.70,0.00,0.0,0.0,0.0,...,NaN,NaN,115.099998,0.0,NaN,0.000000,4.0,2920mAH,16GB/32GB,5.0
36394,13838040676,1,1.0,NaN,河南郑州,143.20,0.20,135.0,9951.0,0.0,...,NaN,1.0,148.810003,0.0,0.200000,135.000000,4.0,3000mAh,32GB,5.5
79629,15945389676,1,0.0,30.0,黑龙江牡丹江,60.99,1.00,90.0,0.0,0.0,...,NaN,NaN,60.990002,0.0,1.000000,90.000000,4.0,3000mAh,16GB,5.7
40228,13888246777,1,1.0,NaN,云南昆明,258.00,0.00,0.0,0.0,0.0,...,NaN,NaN,258.000000,0.0,NaN,0.000000,3.0,1715mAh,16GB/64GB/128GB,4.7
82230,15980726225,1,1.0,NaN,福建福州,78.00,0.00,62.0,0.0,0.0,...,35.0,NaN,91.333333,0.0,NaN,75.333333,4.0,2300mAh,16GB,5.0
62550,15217163366,1,2.0,50.0,广东珠海,55.12,0.00,28.0,0.0,0.0,...,NaN,NaN,54.293334,0.0,NaN,28.000000,4.0,3100mAh,8GB,5.5
99664,18811186422,1,1.0,43.0,北京北京,118.00,0.00,118.0,0.0,0.0,...,NaN,NaN,118.224998,0.0,NaN,118.000000,4.0,3000mAh,16GB,5.5
93368,18374069022,1,0.0,44.0,湖南株洲,31.49,0.00,0.0,0.0,0.0,...,NaN,NaN,22.496667,0.0,NaN,0.000000,NaN,NaN,NaN,NaN
13231,13580829297,1,0.0,NaN,广东东莞,178.00,0.00,173.0,0.0,0.0,...,NaN,NaN,176.866669,0.0,NaN,171.333333,4.0,3750mAh,64GB/128GB/256GB,5.5
